In [ ]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu

In [ ]:
# Multi-GPU FAISS with Wikipedia 202307 Index and Recall Evaluation

import os
import time
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
import gc
from sentence_transformers import SentenceTransformer
import faiss

# Check GPU availability
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")
else:
    print("Warning: No GPU found. Running on CPU.")
    num_gpus = 0

In [ ]:
## 1. Memory Monitoring Functions

def get_memory_usage():
    """Get current memory usage"""
    process = psutil.Process()
    ram_gb = process.memory_info().rss / 1024**3

    gpu_gb = 0
    if torch.cuda.is_available():
        # Sum memory across all GPUs for total usage
        total_gpu_mem = 0
        for i in range(torch.cuda.device_count()):
            total_gpu_mem += torch.cuda.memory_allocated(i)
        gpu_gb = total_gpu_mem / 1024**3

    return {'ram_gb': ram_gb, 'gpu_gb': gpu_gb}

def print_memory_status(label=""):
    """Print current memory status"""
    mem = get_memory_usage()
    print(f"{label} Memory - RAM: {mem['ram_gb']:.2f} GB, GPU: {mem['gpu_gb']:.2f} GB")

In [ ]:
## 2. Load Sentence Transformer Model

# Load model for query encoding
model_name = 'nq-distilbert-base-v1'
bi_encoder = SentenceTransformer(model_name)
print(f"Model: {model_name}")
print(f"Embedding dimension: {bi_encoder.get_sentence_embedding_dimension()}")


In [ ]:
## 3. Recall Evaluation Functions

def calculate_recall_at_k(retrieved_indices, relevant_indices, k=5):
    """Calculate recall@k for retrieved results"""
    if len(relevant_indices) == 0:
        return 1.0 if len(retrieved_indices) == 0 else 0.0
    
    # Take top k retrieved indices
    top_k_retrieved = retrieved_indices[:k] if len(retrieved_indices) >= k else retrieved_indices
    
    # Count how many relevant documents were retrieved
    relevant_retrieved = set(top_k_retrieved).intersection(set(relevant_indices))
    
    # Recall = relevant retrieved / total relevant
    recall = len(relevant_retrieved) / len(relevant_indices)
    return recall

def format_search_results(query, retrieved_indices, distances, recall_score=None):
    """Format detailed search results for output"""
    results = {
        'query': query,
        'recall_at_5': recall_score if recall_score is not None else 0.0,
        'retrieved_docs': []
    }
    
    for i, (idx, dist) in enumerate(zip(retrieved_indices[:5], distances[:5])):
        results['retrieved_docs'].append({
            'rank': i + 1,
            'index': int(idx),
            'distance': float(dist)
        })
    
    return results

def print_search_results(results):
    """Print formatted search results"""
    print(f"\n{'='*80}")
    print(f"QUERY: {results['query']}")
    if results['recall_at_5'] is not None:
        print(f"RECALL@5: {results['recall_at_5']:.3f}")
    print(f"{'='*80}")
    
    for doc in results['retrieved_docs']:
        print(f"  Rank {doc['rank']}: Index {doc['index']}, Distance: {doc['distance']:.4f}")
    print()


In [ ]:
## 4. Load and Distribute Wikipedia Index Across GPUs

def load_wikipedia_index(index_path="wikipedia-2023-07-faiss-index/wikipedia_202307.index"):
    """Load Wikipedia FAISS index from disk"""
    print(f"Loading Wikipedia index from {index_path}...")
    try:
        cpu_index = faiss.read_index(index_path)
        print(f"Successfully loaded index with {cpu_index.ntotal} vectors")
        print(f"Index dimension: {cpu_index.d}")
        return cpu_index
    except Exception as e:
        print(f"Error loading index: {e}")
        print("Make sure the Wikipedia index file exists at the specified path")
        return None

def distribute_index_to_gpus(cpu_index, num_gpus):
    """Distribute FAISS index across multiple GPUs"""
    if num_gpus == 0:
        print("No GPUs available. Using CPU index.")
        return cpu_index
    
    print(f"\nDistributing index across {num_gpus} GPUs...")
    
    # Create GPU resources for each GPU
    gpu_resources = []
    for i in range(num_gpus):
        res = faiss.StandardGpuResources()
        gpu_resources.append(res)
    
    # Option 1: Use index sharding (split index across GPUs)
    if num_gpus > 1:
        print("Using index sharding for multi-GPU setup...")
        
        # Create a sharded index
        co = faiss.GpuMultipleClonerOptions()
        co.shard = True  # Enable sharding
        
        gpu_index = faiss.index_cpu_to_gpus_list(
            cpu_index, 
            gpus=list(range(num_gpus)),
            co=co
        )
        print(f"Index successfully sharded across {num_gpus} GPUs")
    else:
        # Single GPU case
        print("Using single GPU...")
        res = faiss.StandardGpuResources()
        gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
        print("Index successfully loaded to GPU 0")
    
    return gpu_index

# Load the Wikipedia index
cpu_index = load_wikipedia_index()

if cpu_index is not None:
    print_memory_status("Before GPU distribution")
    
    # Distribute index across available GPUs
    gpu_index = distribute_index_to_gpus(cpu_index, num_gpus)
    
    print_memory_status("After GPU distribution")
    
    # Store both for potential fallback
    wikipedia_index = gpu_index if num_gpus > 0 else cpu_index
    print(f"\nIndex ready for search. Total vectors: {wikipedia_index.ntotal}")


In [ ]:
## 5. Multi-GPU Search with Recall Evaluation

def search_multi_gpu(index, queries, k=5):
    """
    Perform multi-GPU search on FAISS index
    
    Args:
        index: FAISS index (CPU or GPU)
        queries: List of query strings or single query string
        k: Number of nearest neighbors to retrieve
    
    Returns:
        Dictionary with search results and metrics
    """
    if isinstance(queries, str):
        queries = [queries]
    
    # Encode queries
    print(f"\nEncoding {len(queries)} queries...")
    query_embeddings = bi_encoder.encode(queries, convert_to_tensor=True)
    
    # Convert to numpy array for FAISS
    if torch.is_tensor(query_embeddings):
        query_embeddings = query_embeddings.cpu().numpy()
    
    # Ensure correct shape
    if len(query_embeddings.shape) == 1:
        query_embeddings = query_embeddings.reshape(1, -1)
    
    # Perform search
    print(f"Searching index with {index.ntotal} vectors...")
    start_time = time.time()
    
    distances, indices = index.search(query_embeddings, k)
    
    search_time = time.time() - start_time
    avg_search_time_ms = (search_time / len(queries)) * 1000
    
    print(f"Search completed in {search_time:.3f} seconds")
    print(f"Average search time per query: {avg_search_time_ms:.2f} ms")
    
    # Format results
    results = {
        'queries': queries,
        'distances': distances,
        'indices': indices,
        'search_time': search_time,
        'avg_search_time_ms': avg_search_time_ms,
        'k': k
    }
    
    return results

# Test queries
test_queries = [
    "What is artificial intelligence?",
    "How does machine learning work?",
    "Explain deep learning algorithms",
    "What are neural networks?",
    "How to implement computer vision?",
    "What is natural language processing?",
    "Explain reinforcement learning",
    "What is supervised learning?",
    "How does unsupervised learning work?",
    "What are transformers in AI?"
]

print("="*80)
print("MULTI-GPU FAISS SEARCH TEST")
print("="*80)

if cpu_index is not None:
    # Perform search
    search_results = search_multi_gpu(wikipedia_index, test_queries, k=10)
    
    # Display results for each query
    print("\n" + "="*80)
    print("SEARCH RESULTS")
    print("="*80)
    
    all_query_results = []
    for i, query in enumerate(search_results['queries']):
        retrieved_indices = search_results['indices'][i]
        distances = search_results['distances'][i]
        
        # Format and store results
        query_result = format_search_results(
            query, 
            retrieved_indices, 
            distances,
            recall_score=None  # No ground truth for Wikipedia dataset
        )
        all_query_results.append(query_result)
        
        # Print results
        print_search_results(query_result)
    
    # Performance summary
    print("\n" + "="*80)
    print("PERFORMANCE SUMMARY")
    print("="*80)
    print(f"Total queries: {len(test_queries)}")
    print(f"Total search time: {search_results['search_time']:.3f} seconds")
    print(f"Average search time per query: {search_results['avg_search_time_ms']:.2f} ms")
    print(f"Index size: {wikipedia_index.ntotal} vectors")
    print(f"Number of GPUs used: {num_gpus}")
    print_memory_status("Final")


In [ ]:
## 6. Advanced Multi-GPU Benchmarking

def benchmark_batch_sizes(index, queries, batch_sizes=[1, 5, 10, 20, 50]):
    """
    Benchmark search performance with different batch sizes
    
    Args:
        index: FAISS index
        queries: List of query strings
        batch_sizes: List of batch sizes to test
    
    Returns:
        DataFrame with benchmark results
    """
    results = []
    
    print("\n" + "="*80)
    print("BATCH SIZE BENCHMARKING")
    print("="*80)
    
    for batch_size in batch_sizes:
        print(f"\nTesting batch size: {batch_size}")
        
        # Select queries for this batch
        batch_queries = queries[:batch_size] if batch_size <= len(queries) else queries * (batch_size // len(queries) + 1)
        batch_queries = batch_queries[:batch_size]
        
        # Run multiple iterations for stable timing
        iterations = 5
        times = []
        
        for iter in range(iterations):
            query_embeddings = bi_encoder.encode(batch_queries, convert_to_tensor=True).cpu().numpy()
            
            start_time = time.time()
            distances, indices = index.search(query_embeddings, 10)
            search_time = time.time() - start_time
            times.append(search_time)
        
        avg_time = np.mean(times)
        std_time = np.std(times)
        throughput = batch_size / avg_time  # queries per second
        
        results.append({
            'batch_size': batch_size,
            'avg_time_s': avg_time,
            'std_time_s': std_time,
            'throughput_qps': throughput,
            'avg_latency_ms': (avg_time / batch_size) * 1000
        })
        
        print(f"  Average time: {avg_time:.4f}s ± {std_time:.4f}s")
        print(f"  Throughput: {throughput:.2f} queries/second")
        print(f"  Average latency: {(avg_time / batch_size) * 1000:.2f} ms/query")
    
    return pd.DataFrame(results)

# Run batch size benchmarking if index is available
if cpu_index is not None:
    benchmark_df = benchmark_batch_sizes(wikipedia_index, test_queries)
    
    print("\n" + "="*80)
    print("BENCHMARK RESULTS SUMMARY")
    print("="*80)
    print(benchmark_df.to_string(index=False))
    
    # Plot results
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Throughput plot
    axes[0].plot(benchmark_df['batch_size'], benchmark_df['throughput_qps'], 'b-o')
    axes[0].set_xlabel('Batch Size')
    axes[0].set_ylabel('Throughput (queries/sec)')
    axes[0].set_title('Search Throughput vs Batch Size')
    axes[0].grid(True)
    
    # Latency plot
    axes[1].plot(benchmark_df['batch_size'], benchmark_df['avg_latency_ms'], 'r-o')
    axes[1].set_xlabel('Batch Size')
    axes[1].set_ylabel('Average Latency (ms/query)')
    axes[1].set_title('Search Latency vs Batch Size')
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()


In [ ]:
## 7. Alternative Multi-GPU Configurations

def test_gpu_configurations(cpu_index):
    """
    Test different GPU configurations (replicated vs sharded)
    
    Args:
        cpu_index: CPU FAISS index
    
    Returns:
        Dictionary with configuration test results
    """
    if num_gpus < 2:
        print("Need at least 2 GPUs for configuration comparison")
        return None
    
    configurations = {}
    
    print("\n" + "="*80)
    print("TESTING DIFFERENT GPU CONFIGURATIONS")
    print("="*80)
    
    # Configuration 1: Sharded (data distributed across GPUs)
    print("\n1. SHARDED CONFIGURATION (Data split across GPUs)")
    print("-" * 40)
    
    co_shard = faiss.GpuMultipleClonerOptions()
    co_shard.shard = True
    
    start_time = time.time()
    gpu_index_sharded = faiss.index_cpu_to_gpus_list(
        cpu_index,
        gpus=list(range(num_gpus)),
        co=co_shard
    )
    shard_setup_time = time.time() - start_time
    
    print(f"  Setup time: {shard_setup_time:.2f} seconds")
    print(f"  Effective index size per GPU: ~{cpu_index.ntotal // num_gpus:,} vectors")
    
    # Test search on sharded index
    query_embedding = bi_encoder.encode(test_queries[0], convert_to_tensor=True).cpu().numpy().reshape(1, -1)
    
    start_time = time.time()
    distances, indices = gpu_index_sharded.search(query_embedding, 10)
    shard_search_time = time.time() - start_time
    
    print(f"  Single query search time: {shard_search_time*1000:.2f} ms")
    
    configurations['sharded'] = {
        'setup_time': shard_setup_time,
        'search_time_ms': shard_search_time * 1000,
        'memory_per_gpu': cpu_index.ntotal // num_gpus
    }
    
    # Configuration 2: Replicated (full index on each GPU, for comparison)
    print("\n2. REPLICATED CONFIGURATION (Full index on each GPU)")
    print("-" * 40)
    
    co_replicate = faiss.GpuMultipleClonerOptions()
    co_replicate.shard = False
    
    start_time = time.time()
    gpu_index_replicated = faiss.index_cpu_to_gpus_list(
        cpu_index,
        gpus=list(range(num_gpus)),
        co=co_replicate
    )
    replicate_setup_time = time.time() - start_time
    
    print(f"  Setup time: {replicate_setup_time:.2f} seconds")
    print(f"  Index size per GPU: {cpu_index.ntotal:,} vectors (full replication)")
    
    # Test search on replicated index
    start_time = time.time()
    distances, indices = gpu_index_replicated.search(query_embedding, 10)
    replicate_search_time = time.time() - start_time
    
    print(f"  Single query search time: {replicate_search_time*1000:.2f} ms")
    
    configurations['replicated'] = {
        'setup_time': replicate_setup_time,
        'search_time_ms': replicate_search_time * 1000,
        'memory_per_gpu': cpu_index.ntotal
    }
    
    # Compare configurations
    print("\n" + "="*80)
    print("CONFIGURATION COMPARISON")
    print("="*80)
    
    comparison_df = pd.DataFrame(configurations).T
    print(comparison_df.to_string())
    
    print("\nRecommendation:")
    if shard_search_time < replicate_search_time:
        speedup = replicate_search_time / shard_search_time
        print(f"  ✓ Sharded configuration is {speedup:.2f}x faster for search")
        print(f"  ✓ Uses {num_gpus}x less memory per GPU")
        print("  → RECOMMENDED for large datasets")
    else:
        speedup = shard_search_time / replicate_search_time
        print(f"  ✓ Replicated configuration is {speedup:.2f}x faster for search")
        print("  ✓ Better for high query throughput with smaller indices")
        print("  → Consider based on your use case")
    
    return configurations

# Test different configurations if we have multiple GPUs
if cpu_index is not None and num_gpus >= 2:
    gpu_configs = test_gpu_configurations(cpu_index)


In [ ]:
## 8. Recall Evaluation with Synthetic Ground Truth (Optional)

def create_synthetic_ground_truth(queries, index_size, relevant_per_query=100):
    """
    Create synthetic ground truth for recall evaluation
    
    Args:
        queries: List of query strings
        index_size: Total size of the index
        relevant_per_query: Number of relevant documents per query
    
    Returns:
        Dictionary mapping queries to relevant document indices
    """
    np.random.seed(42)  # For reproducibility
    ground_truth = {}
    
    for query in queries:
        # Generate random relevant indices for each query
        relevant_indices = np.random.choice(
            index_size, 
            size=min(relevant_per_query, index_size),
            replace=False
        )
        ground_truth[query] = relevant_indices.tolist()
    
    return ground_truth

def evaluate_recall_with_ground_truth(search_results, ground_truth, k_values=[1, 5, 10]):
    """
    Evaluate recall at different k values
    
    Args:
        search_results: Search results from search_multi_gpu
        ground_truth: Dictionary mapping queries to relevant indices
        k_values: List of k values to evaluate
    
    Returns:
        DataFrame with recall metrics
    """
    recall_metrics = []
    
    for i, query in enumerate(search_results['queries']):
        if query not in ground_truth:
            continue
        
        retrieved_indices = search_results['indices'][i]
        relevant_indices = ground_truth[query]
        
        for k in k_values:
            recall_at_k = calculate_recall_at_k(retrieved_indices, relevant_indices, k)
            recall_metrics.append({
                'query': query,
                'k': k,
                'recall': recall_at_k,
                'relevant_docs': len(relevant_indices),
                'retrieved_docs': min(k, len(retrieved_indices))
            })
    
    # Create DataFrame and compute averages
    metrics_df = pd.DataFrame(recall_metrics)
    
    # Compute average recall for each k
    avg_recall = metrics_df.groupby('k')['recall'].mean().reset_index()
    avg_recall.columns = ['k', 'avg_recall']
    
    return metrics_df, avg_recall

# Example: Evaluate with synthetic ground truth
if cpu_index is not None:
    print("\n" + "="*80)
    print("RECALL EVALUATION WITH SYNTHETIC GROUND TRUTH")
    print("="*80)
    
    # Create synthetic ground truth
    synthetic_gt = create_synthetic_ground_truth(
        test_queries, 
        wikipedia_index.ntotal,
        relevant_per_query=1000
    )
    
    print(f"Created synthetic ground truth for {len(synthetic_gt)} queries")
    print(f"Each query has {len(list(synthetic_gt.values())[0])} relevant documents")
    
    # Evaluate recall
    detailed_metrics, avg_recall = evaluate_recall_with_ground_truth(
        search_results,
        synthetic_gt,
        k_values=[1, 5, 10]
    )
    
    print("\n" + "-"*40)
    print("AVERAGE RECALL AT DIFFERENT K VALUES")
    print("-"*40)
    print(avg_recall.to_string(index=False))
    
    # Plot recall curve
    plt.figure(figsize=(8, 5))
    plt.plot(avg_recall['k'], avg_recall['avg_recall'], 'b-o', linewidth=2, markersize=8)
    plt.xlabel('K (Number of Retrieved Documents)')
    plt.ylabel('Average Recall')
    plt.title('Recall@K Curve for Multi-GPU FAISS Search')
    plt.grid(True, alpha=0.3)
    plt.ylim([0, max(avg_recall['avg_recall']) * 1.1])
    
    # Add value labels
    for k, recall in zip(avg_recall['k'], avg_recall['avg_recall']):
        plt.annotate(f'{recall:.3f}', 
                    xy=(k, recall), 
                    xytext=(5, 5), 
                    textcoords='offset points')
    
    plt.tight_layout()
    plt.show()
    
    print("\nNote: These are synthetic ground truth values for demonstration.")
    print("In production, use actual relevance labels for meaningful recall evaluation.")


In [ ]:
## 9. Resource Cleanup and Summary

def cleanup_gpu_resources():
    """Clean up GPU resources"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("GPU cache cleared")
    
    # Force garbage collection
    gc.collect()
    print("Memory cleanup completed")

# Final summary
print("\n" + "="*80)
print("MULTI-GPU FAISS IMPLEMENTATION SUMMARY")
print("="*80)

print("\n📊 KEY FEATURES IMPLEMENTED:")
print("  ✓ Multi-GPU FAISS index distribution (sharding)")
print("  ✓ Wikipedia 202307 index loading and search")
print("  ✓ Batch search optimization")
print("  ✓ Recall@K evaluation framework")
print("  ✓ Performance benchmarking")
print("  ✓ GPU configuration comparison (sharded vs replicated)")

if cpu_index is not None:
    print(f"\n📈 PERFORMANCE METRICS:")
    print(f"  • Index size: {wikipedia_index.ntotal:,} vectors")
    print(f"  • Embedding dimension: {cpu_index.d}")
    print(f"  • Number of GPUs utilized: {num_gpus}")
    print(f"  • Average search latency: {search_results['avg_search_time_ms']:.2f} ms/query")
    
    if num_gpus > 0:
        print(f"\n🔧 GPU CONFIGURATION:")
        print(f"  • Configuration: Sharded (distributed) index")
        print(f"  • Vectors per GPU: ~{wikipedia_index.ntotal // max(1, num_gpus):,}")
        print(f"  • Memory efficiency: {num_gpus}x reduction vs replication")

print("\n💡 RECOMMENDATIONS:")
print("  1. Use sharded configuration for large indices (>1M vectors)")
print("  2. Batch queries for better throughput (10-20 queries optimal)")
print("  3. Monitor GPU memory usage for scaling decisions")
print("  4. Use recall evaluation to tune search parameters")

print("\n🎯 NEXT STEPS:")
print("  • Implement index updates and incremental indexing")
print("  • Add query result caching for frequently accessed queries")
print("  • Integrate with production serving framework")
print("  • Set up continuous monitoring and alerting")

# Cleanup
print("\n🧹 Cleaning up resources...")
cleanup_gpu_resources()
print_memory_status("Final after cleanup")

print("\n✅ Multi-GPU FAISS implementation completed successfully!")
